# SQL проект сервис для чтения книг по подписке

**Цель исследования:**

Сформулировать ценностное предложение для нового продукта

**Задача**

Проанализировать базу данных .  В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг

**Ход исследования**

[1 Подключение к базе данных](#connect)

[2 Исследование  таблицы— вывод первых строк](#limit)

[3	Посчитать, сколько книг вышло после 1 января 2000 года](#step1)

[4	Для каждой книги посчитать количество обзоров и среднюю оценку](#step2)

[5	Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так  исключим из анализа брошюры](#step3)

[6	Определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками](#step4)

[7	Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.](#step5)

[8 выводы ](#text)

## Подключение к базе данных
<a id="connect"></a> 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

""


In [2]:
def read(query):
    result = pd.io.sql.read_sql(query, con = engine)
    return result

## Исследование  таблицы— вывод первых строк
<a id="limit"></a> 

In [3]:
books ='''SELECT * FROM books 
          ORDER BY publication_date ::date 
          LIMIT 5'''

read(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,49,569,A Streetcar Named Desire,107,1952-12-01,97
1,823,20,The Philosophy of Andy Warhol (From A to B and...,272,1977-04-06,178
2,869,542,The Shining,659,1980-07-01,190
3,321,197,Homage to Catalonia,232,1980-10-22,124
4,126,173,Brideshead Revisited,351,1982-01-30,31


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги PK;
- `author_id` — идентификатор автораFK;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателяFK.

In [4]:
authors ='''SELECT  * FROM authors LIMIT 5'''

read(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [5]:
publishers ='''SELECT * FROM publishers LIMIT 5'''

read(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [6]:
ratings ='''SELECT * FROM ratings LIMIT 5'''

read(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [7]:
reviews ='''SELECT * FROM reviews LIMIT 5'''

read(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Посчитать, сколько книг вышло после 1 января 2000 года
<a id="step1"></a> 

In [8]:
count_books_after_2000 ='''SELECT COUNT(book_id) AS Книги_после_1_января_2000_года
                           FROM books
                           WHERE publication_date > '2000-01-01'
                           ; '''
read(count_books_after_2000)

,Книги_после_1_января_2000_года
0,819


В базе содержится 819 книг, которые вышли с 1 января 2000 года

## Для каждой книги посчитать количество обзоров и среднюю оценку
<a id="step2"></a> 

In [9]:
review_avg_rating ='''SELECT bk.title AS Книга,
                             count_review AS Количество_обзоров,
                             avg_rating AS Средний_рейтинг
                      FROM  (SELECT b.book_id,
                                    b.title,
                                    COUNT(rw.review_id) AS count_review
                             FROM books AS b
                             LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
                             
                             GROUP BY b.book_id) AS bk
                      LEFT JOIN (SELECT b.book_id,
                                        b.title,
                                        ROUND(AVG(rng.rating),2) AS avg_rating
                                 FROM books AS b
                                 LEFT JOIN ratings AS rng ON b.book_id = rng.book_id
                                 GROUP BY b.book_id) AS rat ON bk.book_id=rat.book_id
                      ORDER BY count_review DESC
                      ;'''
read(review_avg_rating)

,Книга,Количество_обзоров,Средний_рейтинг
0,Twilight (Twilight #1),7,3.66
1,The Curious Incident of the Dog in the Night-Time,6,4.08
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,The Da Vinci Code (Robert Langdon #2),6,3.83
4,The Glass Castle,6,4.21
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
996,The Natural Way to Draw,0,3.00
997,Essential Tales and Poems,0,4.00
998,Leonardo's Notebooks,0,4.00


*Больше всего обзоров написано на книгу Сумерки, но средний рейтинг книги сильно средний 3.66*

*В то время как на книгу Гарри Поттер и узник Азкабана написано 6 обзоров, а средний рейтинг 4.41*


## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так  исключим из анализа брошюры
<a id="step3"></a> 

In [10]:
publishers_best = ''' SELECT publisher AS Издательство,
                             COUNT(book_id) AS Количество_книг
                      FROM publishers AS p
                      FULL OUTER JOIN books AS b ON p.publisher_id = b.publisher_id
                      WHERE num_pages > 50 
                      GROUP BY p.publisher_id  
                      ORDER BY COUNT(book_id) DESC
                      LIMIT 5;'''
print('Топ-5 издательств')
read(publishers_best)

Топ-5 издательств


,Издательство,Количество_книг
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


*У нас в базе 334 издательства, которые выпускают книги*

## Определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками
<a id="step4"></a> 

In [11]:
author_best = ''' SELECT a.author AS Автор,
                         AVG(rio.rating_avg) AS Средний_рейтинг                               
                  FROM authors a
                  RIGHT JOIN (SELECT rng.book_id AS book_id,
                                     b.author_id AS author_id,
                                     COUNT(rng.rating_id) AS ratings_sum,
                                     AVG(rng.rating) AS rating_avg
                              FROM ratings rng
                              LEFT JOIN books b ON b.book_id = rng.book_id
                              GROUP BY rng.book_id, b.author_id
                              HAVING COUNT(rating_id) >= 50) AS rio ON a.author_id=rio.author_id
                  GROUP BY a.author
                  ORDER BY AVG(rio.rating_avg) DESC
                  LIMIT 5                 
                  ;'''
print('Топ - 5 авторов с самым высоким средним рейтингом')
read(author_best)

Топ - 5 авторов с самым высоким средним рейтингом


,Автор,Средний_рейтинг
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


*Оттобрали 5 авторов с самым высоким средним рейтингом набравшим не менее 50 оценок.* 

*Дж.К. Роулинг/Мэри ГрандПре имеет лучший рейтинг*

## Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок
<a id="step5"></a> 

In [12]:
R= '''SELECT AVG(c_r.count_review) AS Среднее_количество_обзоров
      FROM (SELECT username,
                   COUNT(review_id) AS count_review
            FROM reviews 
            WHERE username IN (SELECT username
                               FROM ratings
                               GROUP BY username
                               HAVING COUNT(rating_id) > 48)
            GROUP BY username) AS c_r
      ;
                    '''
read(R)

,Среднее_количество_обзоров
0,24.0


*Активные пользователи пишут в среднем 24 обзора* 

## Выводы
<a id="text"></a> 

В базе хранятся книги с 1952 года. С 1 января 2000 года вышло 819 книг 80% от общего количества 1000шт. 

Больше всего обзоров написано на книгу Сумерки, но средний рейтинг книги сильно средний 3.66

В то время как на книгу Гарри Поттер и узник Азкабана написано 6 обзоров, а средний рейтинг 4.41

У нас в базе 334 издательства

Больше всего книг в базе представлено издательством Penguin Books 42 шт.

Книги автора  J.K. Rowling/Mary GrandPré  читалелям нравятся больше всего

В среднем активные пользователи,  которые оценили более 48 книг пишут 24 образа
